In [7]:
from skimage.color import rgb2gray
import skimage.io as io
import numpy as np
import cv2 as cv2


In [2]:
vidcap = cv2.VideoCapture('LaneVideo.mp4')
success, image = vidcap.read()

def nothing(x):
    pass

cv2.namedWindow("Trackbars")

cv2.createTrackbar("L - H", "Trackbars", 0, 255, nothing)
cv2.createTrackbar("L - S", "Trackbars", 0, 255, nothing)
cv2.createTrackbar("L - V", "Trackbars", 200, 255, nothing)
cv2.createTrackbar("U - H", "Trackbars", 255, 255, nothing)
cv2.createTrackbar("U - S", "Trackbars", 50, 255, nothing)
cv2.createTrackbar("U - V", "Trackbars", 255, 255, nothing)

while success:
    success, image = vidcap.read()
    frame = cv2.resize(image,(640,480))

    ## Choosing points for perspective transformation
    tl = (222,387)
    bl = (70 ,472)
    tr = (400,380)
    br = (538,472)

    cv2.circle(frame, tl, 5, (0,0,255), -1)
    cv2.circle(frame, bl, 5, (0,0,255), -1)
    cv2.circle(frame, tr, 5, (0,0,255), -1)
    cv2.circle(frame, br, 5, (0,0,255), -1)

    ## Aplying perspective transformation
    pts1 = np.float32([tl, bl, tr, br]) 
    pts2 = np.float32([[0, 0], [0, 480], [640, 0], [640, 480]]) 
    
    # Matrix to warp the image for birdseye window
    matrix = cv2.getPerspectiveTransform(pts1, pts2) 
    transformed_frame = cv2.warpPerspective(frame, matrix, (640,480))

    ### Object Detection
    # Image Thresholding
    hsv_transformed_frame = cv2.cvtColor(transformed_frame, cv2.COLOR_BGR2HSV)
    
    l_h = cv2.getTrackbarPos("L - H", "Trackbars")
    l_s = cv2.getTrackbarPos("L - S", "Trackbars")
    l_v = cv2.getTrackbarPos("L - V", "Trackbars")
    u_h = cv2.getTrackbarPos("U - H", "Trackbars")
    u_s = cv2.getTrackbarPos("U - S", "Trackbars")
    u_v = cv2.getTrackbarPos("U - V", "Trackbars")
    
    lower = np.array([l_h,l_s,l_v])
    upper = np.array([u_h,u_s,u_v])
    mask = cv2.inRange(hsv_transformed_frame, lower, upper)


error: OpenCV(4.8.1) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4062: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'


: 

In [9]:
def Get_lanes_bases(img):

    # Get the Histogram of the bottom half of the frame captured
    half_frame = img.shape[0] //2

    crop_half_frame = img[half_frame:,:]
    axis_Histogram = np.sum(crop_half_frame,axis = 0)

    # Get where is the max values located on the x-axis
    left_lane = np.argmax(axis_Histogram[:img.shape[1]//2])
    right_lane = (img.shape[1]//2) + np.argmax(axis_Histogram[img.shape[1]//2:]) 

    return left_lane, right_lane


In [8]:
def sliding_window(img, imgHeight, left_lane, right_lane, window_height, window_width):
    
    # Arrays to hold the lane index
    right_lane_arr = []
    left_lane_arr = []

    left_y = []
    right_y = []
    
    # Creating the window on the lanes bases
    num_of_windows = imgHeight//window_height

    # Move the window on the lane
    for i in range(num_of_windows):
        left_lane_win = img[imgHeight - window_height:imgHeight, left_lane - window_width/2:left_lane + window_width/2]
        right_lane_win = img[imgHeight - window_height:imgHeight, right_lane - window_width/2:right_lane + window_width/2]

        # get the white pixels in the left and right window
        left_non_zero = np.argwhere(left_lane_win = 1)
        right_non_zero = np.argwhere(right_lane_win = 1)

        # add the index to the lane arrays
        left_new_index = left_non_zero[:,1] + (left_lane - window_width//2)
        right_new_index = right_non_zero[:,1] + (right_lane - window_width//2)

        left_lane_arr = np.append(left_new_index)
        right_lane_arr = np.append(right_new_index)

        # Lanes Y index
        left_y[i],right_y[i] = imgHeight - i*window_height

        # update the window base
        left_lane = np.mean(left_new_index)
        right_lane = np.mean(right_new_index)

    # Curve fitting(2nd degree poynomial) for the points from the window
    left_curve = np.polyfit(left_y, left_lane_arr, 2)
    right_curve = np.polyfit(right_y, right_lane_arr, 2)

    return left_curve, right_curve


In [ ]:
def get_Curvature(left_curve, right_curve):

    # polynomial fitting = Ax^2 + Bx + C
    A_left, B_left, C_left = left_curve
    A_right, B_right, C_right = right_curve

    # Radius of curvature = ((1 + (2Ax + B)**2)**(3/2)) / |2A|
    


    